Bibliotecas necessárias

In [164]:
# importação das bibliotecas necessárias 
import bpy # Blender Python
# import bpy_extras 
import os # Sistema Operacional
import mathutils # Manipulação matricial
import math # Funções matemáticas
from tqdm import tqdm # Barra de progresso
import contextlib #suprimir prints do blender
import cv2 as cv #
import numpy as np
from pathlib import Path

Input do usuário

In [165]:
######################## INPUT ###########################
#Please write de arquive name '.blend'. Attention to the instructions above
object_name = "Bunny"
epsilon = 0.01
resolution_images = 1920
base_folder = "C:/Users/Daniel/OneDrive/Área de Trabalho/TCC/TCC/planes_intersections"

######################## INPUT ###########################

Funções auxiliares

In [166]:
#Função que cria um plano fatiador no arquivo 
def create_cutter_plane(location, size=(100,100,100)):
    bpy.ops.mesh.primitive_plane_add(size=1, enter_editmode=False, align='WORLD', location=location, rotation=(0,0,0))
    plane = bpy.context.object
    plane.scale = size
    return plane

#Função que aplica o modificadorbooleano (tira as intersecções)
def apply_boolean_modifier(cutter, obj):
    mod = obj.modifiers.new(name='Boolean_Intersection', type='BOOLEAN')
    mod.operation = 'INTERSECT'
    mod.object = cutter
    mod.solver = 'EXACT'

    mod.use_self = True  # Ativa Self Intersection

    # Aplicar o modificador
    bpy.context.view_layer.objects.active = obj
    bpy.ops.object.modifier_apply(modifier=mod.name)

#Função que renderiza a imagem
def renderizar(contador):
        arquivo = f"{object_name}_{contador}.png"
        bpy.context.scene.render.filepath = os.path.join(output_folder, arquivo)
        bpy.ops.render.render(write_still=True)

#Função que retorna a dimensão do objeto (by bounding_box)
def obter_dimensao(obj, axis = None):
    # Obter os vértices da bounding box do objeto
    bbox_corners = [obj.matrix_world @ mathutils.Vector(corner) for corner in obj.bound_box]
    if axis and axis.lower() == "z":
        dimensao = max([corner[2] for corner in bbox_corners]) - min([corner[2] for corner in bbox_corners])
        print("Coletada dimensão z:", dimensao)
    else:
        # Calcular as coordenadas mínimas e máximas ao longo do eixo X
        min_x = min(v.x for v in bbox_corners)
        max_x = max(v.x for v in bbox_corners)

        # Calcular as coordenadas mínimas e máximas ao longo do eixo Y
        min_y = min(v.y for v in bbox_corners)
        max_y = max(v.y for v in bbox_corners)
        
        # Calcular a dimensão como max entre dimensao x e y
        print("Dimensão x: ", max_x - min_x)
        print("Dimensão y: ", max_y - min_y)
        dimensao = max(max_x - min_x, max_y - min_y)

    return dimensao

#Função que modifica a cor de um objeto
def set_color(objeto, color=(0, 0, 0, 1)):
    if not hasattr(objeto, 'data') or objeto.data is None:
        print("Erro: O objeto não possui dados de malha.")
        return

    if not hasattr(objeto.data, 'materials'):
        print("Erro: O objeto não suporta materiais.")
        return

    # Limpa os materiais existentes
    while objeto.data.materials:
        objeto.data.materials.pop(index=0)

    # Cria um novo material
    mat = bpy.data.materials.new(name="Color")
    objeto.data.materials.append(mat)

    # Configura o material
    mat.use_nodes = True
    bsdf = mat.node_tree.nodes.get("Principled BSDF")
    if bsdf:
        bsdf.inputs['Base Color'].default_value = color
    else:
        print("Erro: Nó Principled BSDF não encontrado.")

    objeto.active_material = mat
    # print(f"Material configurado com cor: {color}")

#Função que encontra o ponto central do objeto
def center_obj(obj):
    # Obter os vértices da bounding box do objeto
    bbox_corners = [obj.matrix_world @ mathutils.Vector(corner) for corner in obj.bound_box]

    # Calcular o centro geométrico (média dos vértices da bounding box)
    centro_x = sum([v.x for v in bbox_corners]) / 8
    centro_y = sum([v.y for v in bbox_corners]) / 8
    centro_z = sum([v.z for v in bbox_corners]) / 8
    
    return centro_x, centro_y, centro_z


#Função que inibi plots automáticos do blender
class NullWriter:
    def write(self, _):
        pass

def add_thickness_to_plane(plane, thickness=0.01):
    # Adicionar o modificador Solidify
    solidify_mod = plane.modifiers.new(name="Solidify", type='SOLIDIFY')
    solidify_mod.thickness = thickness

    # Aplicar o modificador
    bpy.context.view_layer.objects.active = plane
    bpy.ops.object.modifier_apply(modifier=solidify_mod.name)


In [167]:
######################### Reconhecimento do objeto tridimensional ######################### 
# Caminho até o arquivo
filepath = 'blender_arquives\\{}.blend'.format(object_name)
# Verifica se o arquivo existe
if os.path.exists(filepath):
    bpy.ops.wm.open_mainfile(filepath=filepath)
else:
    raise FileNotFoundError(f"Erro: O arquivo '{object_name}.blend' não existe.")

# Tentar obter o objeto pelo nome
obj = bpy.data.objects.get(object_name)
if obj is None:
    raise ValueError(f"Objeto '{object_name}' não encontrado no arquivo {object_name}.blend")
else: 
    print("Nome do objeto reconhecido: " + object_name)

######################### Informações do objeto ######################### 
# Selecionar o objeto
bpy.context.view_layer.objects.active = obj
obj.select_set(True)
# Obtenha as coordenadas do bounding box como vetores
bounding_box_corners = [obj.matrix_world @ mathutils.Vector(corner) for corner in obj.bound_box]
# Calcule as dimensões a partir do bounding box
dimension = obter_dimensao(obj,'z')
# print("Localização do objeto:", obj.location)

center_x, center_y, center_z = center_obj(obj)

# Calcule os limites inferior e superior
min_coord = center_z - (dimension / 2)
max_coord = center_z + (dimension / 2)
print("Dimensão z do objeto: ", dimension)
print("Centro do objeto: ", center_z)

######################### Quantidade de fatias ######################### 
# Calcular o número de fatias
num_slices = math.floor( dimension / float(epsilon))
print("Número de fatias: ", num_slices)

######################### Posicionamento da câmera ######################### 
# Posicionar a câmera acima
bpy.ops.object.camera_add(enter_editmode=False, align='VIEW', location=(0, 0, max_coord + 1))
camera = bpy.context.object
bpy.context.scene.camera = camera
camera_position = camera.location

# Definir a resolução quadrada (por exemplo, 1080x1080 pixels)
bpy.context.scene.render.resolution_x = resolution_images  # Largura
bpy.context.scene.render.resolution_y = resolution_images  # Altura
bpy.context.scene.render.resolution_percentage = 100  # Garantir que a resolução seja 100%

# FOV da câmera
fov_x = camera.data.angle_x
fov_y = camera.data.angle_y

######################### Conversão unidade -> pixel ######################### 
# Distância da câmera até a altura inicial
distance_to_h = abs(camera_position.z - min_coord)

#Remove a câmera criada
bpy.data.objects.remove(camera, do_unlink=True)

distance = obter_dimensao(obj)
distance += 1
epsilon_pixels = round((epsilon*(resolution_images))/distance, 5)
print(f"Epsilon em pixels: ", epsilon_pixels)


######################### Adicionando contraste ao objeto ######################### 
#Muda a cor do objeto para preto
# Após adicionar ou remover objetos, garanta que o objeto original seja reativado
bpy.context.view_layer.objects.active = obj
obj.select_set(True)

if obj:
    print(f"Objeto selecionado: {obj.name}")
    print(f"Tipo do objeto: {obj.type}")
else:
    print("Erro: Nenhum objeto está ativo ou selecionado.")

set_color(obj, (0,0,0,1))

######################### Fecha o arquivo ######################### 
cont=0
# Suprimir as mensagens de saída
with contextlib.redirect_stdout(NullWriter()):
    # Atualiza a visualização
    bpy.context.view_layer.update()

    # Salva a atualização de cor
    bpy.ops.wm.save_mainfile()

    # Fecha o Blender
    bpy.ops.wm.quit_blender()
    cont = 10

Nome do objeto reconhecido: Bunny
Coletada dimensão z: 1.0263184607028961
Dimensão z do objeto:  1.0263184607028961
Centro do objeto:  0.5433173626661301
Número de fatias:  102
Dimensão x:  1.0353982746601105
Dimensão y:  0.8024774789810181
Epsilon em pixels:  9.43304
Objeto selecionado: Bunny
Tipo do objeto: MESH


Preparar a pasta destino que vai receber as imagens

In [168]:
#Se não existir a pasta 'base_folder', cria
if not os.path.exists(base_folder):
    os.makedirs(base_folder)

#Define o diretório de saída
output_folder = os.path.join(base_folder, object_name + "_{}".format(epsilon_pixels))

# Prepara o diretório de saída. Se não existir, cria. Se existir, limpa.
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
else:
    # Limpa o diretório de saída
    for file in os.listdir(output_folder):
        file_path = os.path.join(output_folder, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)
print(f"Diretório de saída pronto: ../planes_intersections/{object_name}_{epsilon_pixels}")

Diretório de saída pronto: ../planes_intersections/Bunny_9.43304


Fatiamento do objeto

In [169]:
######################### Configurando a câmera inicial ######################### 
#Abro o arquivo .blender
bpy.ops.wm.open_mainfile(filepath='blender_arquives\\{}.blend'.format(object_name))

#Calcula a altura em que a câmera deve ficar inicialmente
h = distance / (2 * math.tan(fov_x / 2))

for i in tqdm(range(0, num_slices), desc="Fatiando objeto", ncols=100):
        # Tentar obter o objeto pelo nome
        obj = bpy.data.objects.get(object_name)

        #Determino a localização em que o plano vai estar
        loc = [center_x, center_y, min_coord + (i)*epsilon]
        # print(loc)
        # Criar o plano de corte
        cutter = create_cutter_plane(location=loc)
        set_color(cutter, color=(0,0,0,1))

        # Posicionar a câmera acima do objeto
        altura_camera = min_coord + (i)*epsilon + h
        bpy.ops.object.camera_add(enter_editmode=False, align='VIEW', location=(loc[0],loc[1],altura_camera))
        camera = bpy.context.object
        bpy.context.scene.camera = camera

        # Adicionar uma luz junto à câmera
        bpy.ops.object.light_add(type='POINT', location=camera.location)
        light = bpy.context.object
        light.data.energy = 100  # Intensidade mínima da luz


        # Aplicar o modificador booleano
        apply_boolean_modifier(obj, cutter)

        #limito a visualização inferior do objeto
        cutter_under = create_cutter_plane(location=(loc[0],loc[1], loc[2]-epsilon/100))
        set_color(cutter_under, color=(1, 1, 1, 1))


        #Torna invisível o objeto inteiro
        obj.hide_render = True
        #Tira a foto
        renderizar(i+1)

        #Torna visível obj novamente
        obj.hide_render = False
        

        # Remover a luz após o uso
        bpy.data.objects.remove(light, do_unlink=True)
        #Limpeza do arquivo, retornando a versão original
        bpy.data.objects.remove(camera, do_unlink=True)        
        #Apaga o plano da cena
        bpy.data.objects.remove(cutter, do_unlink=True)
        #Apaga o plano da cena
        bpy.data.objects.remove(cutter_under, do_unlink=True)


Fatiando objeto: 100%|██████████████████████████████████████████| 102/102 [1:49:50<00:00, 64.61s/it]


In [173]:
paste  = f"{object_name}_{epsilon_pixels}"
objeto = object_name
epsilon = epsilon_pixels

#Criação da array
cont_img = 1
altura = 0
pontos_extraidos = []

#caminho até a pasta de imagens
caminho_pasta = Path("C:/Users/Daniel/OneDrive/Área de Trabalho/TCC/TCC/planes_intersections/{}".format(paste))
# Contar o número de arquivos na pasta
numero_arquivos = len(list(caminho_pasta.glob('*')))
#tamanho da imagem
img = cv.imread("planes_intersections\\{}\\{}_{:d}.png".format(paste, objeto, 1))
size_image = img.shape[0]

# print("Extraindo pontos das seções paralelas...")
for i in tqdm(range(1, numero_arquivos + 1), desc="Coletando nuvem de pontos", ncols=100):
        # Leitura da imagem e cálculo dos pontos de borda
        img_path = "planes_intersections\\{}\\{}_{:d}.png".format(paste, objeto, i)
        img = cv.imread(img_path, 0)

        if img is None:
                continue
        else:
            # Aplicação do filtro Sobel e binarização
            sobelx = cv.Sobel(img, cv.CV_64F, 1, 0, ksize=3)
            sobely = cv.Sobel(img, cv.CV_64F, 0, 1, ksize=3)
            sobel = np.hypot(sobelx, sobely)
            _, binary_sobel = cv.threshold(sobel, 50, 255, cv.THRESH_BINARY)
            
                # Suponha que você tenha um loop para escrever no arquivo
            for i in range(binary_sobel.shape[0]):
                    for j in range(binary_sobel.shape[1]):
                        if binary_sobel[i, j] > 0:
                            # Escrever no arquivo apenas se ele ainda estiver aberto
                            #Conversão de escalonamento 
                          pontos_extraidos.append([i / size_image, j / size_image, altura / size_image])
            altura += float(epsilon)

# def subsample_points(points, min_distance):
#     """Função para aplicar subsampling baseado em distância mínima horizontal"""
#     subsampled_points = []
#     last_point = None
#     descartados = 0

#     for point in points:
#         if last_point is None or np.sqrt((point[0] - last_point[0])**2 + (point[1] - last_point[1])**2 + (point[2] - last_point[2])**2) > min_distance:
#             subsampled_points.append(point)
#             last_point = point
#         else:
#             descartados += 1

#     return np.array(subsampled_points), descartados

# print("Filtrando pontos...")
# # Aplicar o filtro de subsampling
# min_distance = float(epsilon)/size_image  # Ajuste a distância mínima conforme necessário
# print(min_distance)
# subsampled_points, descartados = subsample_points(pontos_extraidos, min_distance)

# Arquivo ply
# Caminho do arquivo .ply
new_arquive = 'meshlab_arquives\\pontos_{}.ply'.format(paste)

#se já existir o arquivo new_arquive apaga para refazê-lo
if os.path.isfile(new_arquive):
    os.unlink(new_arquive) 

def save_ply_points(filename, points):
    """Função para salvar pontos em um arquivo .ply"""
    with open(filename, 'w') as f:
        # Cabeçalho do arquivo .ply
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {len(points)}\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("end_header\n")
        
        # Escrever cada ponto no arquivo
        for point in points:
            f.write(f"{point[0]} {point[1]} {point[2]}\n")

print("Gerando arquivo ply...")
# Salvar os pontos subamostrados no arquivo .ply
save_ply_points(new_arquive, pontos_extraidos)

print(f"Pontos subamostrados foram salvos em {new_arquive}")


Coletando nuvem de pontos: 100%|██████████████████████████████████| 101/101 [01:43<00:00,  1.02s/it]


Gerando arquivo ply...
Pontos subamostrados foram salvos em meshlab_arquives\pontos_Bunny_9.43304.ply


In [171]:
print("Pontos totais:", len(pontos_extraidos))
# print("Descartados:", descartados)
print("Pontos restantes", len(pontos_extraidos))

Pontos totais: 345154
Pontos restantes 345154
